# Heterogenous Modelling
Using different kinds of distributions to model data, instead of only a single kind of distribution.


In [17]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_probability as tfp

import sys
sys.path.append('../')
import utils as utl
import datasets as d
import models as m

tfd = tfp.distributions
tfm = tf.math

%load_ext autoreload
%autoreload 2

C:\Users\sap98\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."


## Get Toy Data

In [18]:
N = 1000
data = d.gen_checkerboard_d3split(batch_size=N)
batched = d.to_tf_dataset(data, batch_size=200)

## Define Model

In [ ]:
# Number of components per distribution
K_TT = 5  # 10
K_CP = 18  # 70
# Number of dimensions of the data (features)
M = data.shape[1]

In [ ]:
# Define a distribution for each dimension of the data
dists = [
    tfd.Normal,     # dimension 1
    tfd.Normal,     # dimension 2
    tfd.Categorical # dimension 3
]

In [ ]:
# Define the parameters used by the models
params_TT = [
    [
        # distribution 1 (normal)
        np.random.uniform(-4, 4, (K_TT, K_TT)),
        np.random.uniform(0, 4, (K_TT, K_TT))
    ], [
        # distribution 2 (normal)
        np.random.uniform(-4, 4, (K_TT, K_TT)),
        np.random.uniform(0, 4, (K_TT, K_TT))
    ], [
        # distribution 3 (categorical)
        np.ones((K_TT, K_TT, 3))
    ]
]

params_CP = [
    [
        np.random.uniform(-4, 4, (K_CP, )),
        np.random.uniform(0, 4, (K_CP, ))
    ], [
        np.random.uniform(-4, 4, (K_CP, )),
        np.random.uniform(0, 4, (K_CP, ))
    ], [
        np.ones((K_CP, 3))
    ],
]

In [ ]:
# Define the modifyer used for each dimension
# set to None for categorical
modifiers = {
    0: {1: tfm.softplus},
    1: {1: tfm.softplus}
}

In [ ]:
# Instantiate models
TT = m.TensorTrainGeneral(K_TT, dists, params_TT, modifiers)
CP = m.CPGeneral(K_CP, dists, params_CP, modifiers)

## Fit models to data

In [ ]:
# Define number of epochs to train on
EPOCHS = 250

In [ ]:
# Training optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

In [ ]:
# train TT general version
losses_TT = TT.fit(batched, EPOCHS, optimizer)

In [ ]:
# train TT fixed gaussian version 
losses_CP = CP.fit(batched, EPOCHS, optimizer)

## Display Results

### Loss Plot

In [ ]:
f, ax = plt.subplots(2, 1, figsize=(9,6))
ax[0].plot(losses_TT)
ax[0].set_title("Heterogeneous TT")
ax[0].set_ylim([3.5, 6])
ax[1].plot(losses_CP)
ax[1].set_title("Heterogeneous CP")
ax[1].set_ylim([3.5, 6])
f.tight_layout()
f.add_subplot(111, frameon=False)
# hide tick and tick label of the big axis
plt.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
plt.xlabel("Training Iterations")
plt.ylabel("Negative Log-Likelihood pr. sample")
plt.show()

### Density Plots

In [ ]:
utl.plot_density_3d(CP, 3, "Heterogeneous CP Decomposition")

In [ ]:
utl.plot_density_3d(TT, 3, "Heterogeneous TT Decomposition")

### Sampling

In [ ]:
# sample from the models
N_samples = 1000
TT_samples = TT.sample(N_samples)
CP_samples = CP.sample(N_samples)

In [ ]:
# plot the samples
f, ax = plt.subplots(1, 2, figsize=(12, 4))

ax[0].plot(CP_samples[CP_samples[:, 2] == 0, 0], CP_samples[CP_samples[:, 2] == 0, 1], '.')
ax[0].plot(CP_samples[CP_samples[:, 2] == 1, 0], CP_samples[CP_samples[:, 2] == 1, 1], '.')
ax[0].plot(CP_samples[CP_samples[:, 2] == 2, 0], CP_samples[CP_samples[:, 2] == 2, 1], '.')
ax[0].set_title('Sampling from CP model')

ax[1].plot(CP_samples[CP_samples[:, 2] == 0, 0], CP_samples[CP_samples[:, 2] == 0, 1], '.')
ax[1].plot(CP_samples[CP_samples[:, 2] == 1, 0], CP_samples[CP_samples[:, 2] == 1, 1], '.')
ax[1].plot(CP_samples[CP_samples[:, 2] == 2, 0], CP_samples[CP_samples[:, 2] == 2, 1], '.')
ax[1].set_title('Sampling from TT model')

plt.show()